##### Copyright 2023 Google LLC

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Setup

### Install & import


In [ ]:
!pip install -U -q google-generativeai

In [ ]:
# import necessary modules.
import google.generativeai as genai
import json
import base64
import pathlib
import pprint
import requests
import mimetypes
from IPython.display import Markdown

## Set the API key

Add your API_KEY to the secrets manager in the left pannel "🔑".

In [ ]:
from google.colab import userdata

API_KEY=userdata.get('API_KEY')

In [ ]:
# Configure the client library by providing your API key.
genai.configure(api_key=API_KEY)

### Parse the arguments

In [ ]:
model = 'gemini-1.5-pro-latest' # @param {isTemplate: true}
contents_b64 = 'W10=' # @param {isTemplate: true}
generation_config_b64 = 'eyJ0ZW1wZXJhdHVyZSI6MSwidG9wX3AiOjAuOTUsInRvcF9rIjowLCJtYXhfb3V0cHV0X3Rva2VucyI6ODE5Miwic3RvcF9zZXF1ZW5jZXMiOltdfQ==' # @param {isTemplate: true}
safety_settings_b64 = 'W3siY2F0ZWdvcnkiOiJIQVJNX0NBVEVHT1JZX0hBUkFTU01FTlQiLCJ0aHJlc2hvbGQiOiJCTE9DS19NRURJVU1fQU5EX0FCT1ZFIn0seyJjYXRlZ29yeSI6IkhBUk1fQ0FURUdPUllfSEFURV9TUEVFQ0giLCJ0aHJlc2hvbGQiOiJCTE9DS19NRURJVU1fQU5EX0FCT1ZFIn0seyJjYXRlZ29yeSI6IkhBUk1fQ0FURUdPUllfU0VYVUFMTFlfRVhQTElDSVQiLCJ0aHJlc2hvbGQiOiJCTE9DS19NRURJVU1fQU5EX0FCT1ZFIn0seyJjYXRlZ29yeSI6IkhBUk1fQ0FURUdPUllfREFOR0VST1VTX0NPTlRFTlQiLCJ0aHJlc2hvbGQiOiJCTE9DS19NRURJVU1fQU5EX0FCT1ZFIn1d' # @param {isTemplate: true}
user_input_b64 = 'ZnJvbSBmbGFzayBpbXBvcnQgRmxhc2ssIHJlcXVlc3QsIGpzb25pZnkKaW1wb3J0IGdvb2dsZS5nZW5lcmF0aXZlYWkgYXMgcGFsbQpmcm9tIHNrbGVhcm4ubGluZWFyX21vZGVsIGltcG9ydCBMb2dpc3RpY1JlZ3Jlc3Npb24gICMgRXhlbXBsbyBkZSBtb2RlbG8KaW1wb3J0IHBpY2tsZQoKYXBwID0gRmxhc2soX19uYW1lX18pCgojIENvbmZpZ3VyYXIgYSBBUEkgS2V5IGRhIEdlbWluaQpwYWxtLmNvbmZpZ3VyZShhcGlfa2V5PSdTVUFfQVBJX0tFWScpCgojIENhcnJlZ2FyIG8gbW9kZWxvIGRlIE1hY2hpbmUgTGVhcm5pbmcgdHJlaW5hZG8Kd2l0aCBvcGVuKCdzZW50aW1lbnRfbW9kZWwucGtsJywgJ3JiJykgYXMgZjoKICAgIG1vZGVsID0gcGlja2xlLmxvYWQoZikgCgpAYXBwLnJvdXRlKCcvYW5hbHl6ZScsIG1ldGhvZHM9WydQT1NUJ10pCmRlZiBhbmFseXplX3NlbnRpbWVudCgpOgogICAgZGF0YSA9IHJlcXVlc3QuZ2V0X2pzb24oKQogICAgdGV4dCA9IGRhdGEuZ2V0KCd0ZXh0JykKCiAgICBpZiBub3QgdGV4dDoKICAgICAgICByZXR1cm4ganNvbmlmeSh7J2Vycm9yJzogJ1RleHRvIG7Do28gZm9ybmVjaWRvJ30pLCA0MDAKCiAgICBlbWJlZGRpbmcgPSBnZXJhcl9lbWJlZGRpbmdzKHRleHQpCiAgICBzZW50aW1lbnQgPSBtb2RlbC5wcmVkaWN0KFtlbWJlZGRpbmddKVswXQogICAgY29uZmlkZW5jZSA9IG1heChtb2RlbC5wcmVkaWN0X3Byb2JhKFtlbWJlZGRpbmddKVswXSkKCiAgICByZXR1cm4ganNvbmlmeSh7CiAgICAgICAgJ3NlbnRpbWVudCc6IHNlbnRpbWVudCwKICAgICAgICAnY29uZmlkZW5jZSc6IGNvbmZpZGVuY2UKICAgIH0pCgpkZWYgZ2VyYXJfZW1iZWRkaW5ncyh0ZXh0KToKICAgIHJlc3BvbnNlID0gcGFsbS5nZW5lcmF0ZV9lbWJlZGRpbmdzKG1vZGVsPSdtb2RlbHMvZW1iZWRkaW5nLWdlY2tvJywgdGV4dD10ZXh0KQogICAgcmV0dXJuIHJlc3BvbnNlLmVtYmVkZGluZ3MKCmlmIF9fbmFtZV9fID09ICdfX21haW5fXyc6CiAgICBhcHAucnVuKGRlYnVnPVRydWUp' # @param {isTemplate: true}

contents = json.loads(base64.b64decode(contents_b64))
generation_config = json.loads(base64.b64decode(generation_config_b64))
safety_settings = json.loads(base64.b64decode(safety_settings_b64))
user_input = base64.b64decode(user_input_b64).decode()
stream = False

In [ ]:
contents

[{'role': 'user', 'parts': [{'text': 'hello'}]},
 {'role': 'model', 'parts': [{'text': 'Hello! How can I assist you today?'}]}]

In [ ]:
generation_config

{}

In [ ]:
safety_settings

{}

In [ ]:
user_input

'How does electricity work?'

### Call the API

In [ ]:
# Call the model and print the response.
gemini = genai.GenerativeModel(model_name=model)

chat = gemini.start_chat(history=contents)

response = chat.send_message(
    user_input,
    stream=stream)

In [ ]:
display(Markdown(response.text))

Electricity is the flow of electric charge. It is a fundamental part of nature and is responsible for a wide range of phenomena, from lightning and static electricity to the operation of electronic devices.

**Basic Principles of Electricity:**

1. **Electric Charge:** Matter is made up of atoms, which have a positively charged nucleus surrounded by negatively charged electrons. The movement of these charged particles creates electric currents.

2. **Electric Field:** Electric charges create an electric field around them. The strength and direction of this field depend on the amount and type of charge.

3. **Electric Potential:** Electric potential is the amount of electrical energy stored in a region of space. It is measured in volts (V).

4. **Electric Current:** Electric current is the flow of electric charge through a conductor. It is measured in amperes (A).

**How Electricity Works:**

1. **Generation of Electricity:** Electricity can be generated in various ways, including:
   - **Mechanical Energy:** Generators convert mechanical energy into electrical energy by spinning a conductor in a magnetic field. This is the principle behind power plants that use turbines driven by steam, water, or wind.
   - **Chemical Energy:** Batteries generate electricity through chemical reactions.
   - **Solar Energy:** Solar cells convert sunlight directly into electricity using the photovoltaic effect.

2. **Transmission of Electricity:** Electricity is transmitted over long distances through power lines. These lines are made of conductive materials, such as copper or aluminum, which allow electrons to flow easily.

3. **Distribution of Electricity:** Once electricity reaches populated areas, it is distributed to homes, businesses, and other consumers through a network of local power lines and transformers.

4. **Utilization of Electricity:** Electricity is used to power various devices and appliances by converting electrical energy into other forms of energy, such as light, heat, or motion.

The flow of electricity in a circuit is driven by the difference in electric potential between two points. When a complete circuit is formed, electrons can flow from a region of higher potential to a region of lower potential, creating an electric current.

Electricity is a versatile form of energy that has revolutionized modern society. It is used in a wide range of applications, from lighting and heating to powering computers and transportation systems.

In [ ]:
response.prompt_feedback

safety_ratings {
  category: HARM_CATEGORY_SEXUALLY_EXPLICIT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HATE_SPEECH
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HARASSMENT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_DANGEROUS_CONTENT
  probability: NEGLIGIBLE
}

In [ ]:
response.candidates

[index: 0
content {
  parts {
    text: "Electricity is the flow of electric charge. It is a fundamental part of nature and is responsible for a wide range of phenomena, from lightning and static electricity to the operation of electronic devices.\n\n**Basic Principles of Electricity:**\n\n1. **Electric Charge:** Matter is made up of atoms, which have a positively charged nucleus surrounded by negatively charged electrons. The movement of these charged particles creates electric currents.\n\n2. **Electric Field:** Electric charges create an electric field around them. The strength and direction of this field depend on the amount and type of charge.\n\n3. **Electric Potential:** Electric potential is the amount of electrical energy stored in a region of space. It is measured in volts (V).\n\n4. **Electric Current:** Electric current is the flow of electric charge through a conductor. It is measured in amperes (A).\n\n**How Electricity Works:**\n\n1. **Generation of Electricity:** Electr